In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pytz
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# rule = 'BME'

time_range = '5y'

start_time = '2019-12-31'
end_time = '2024-01-01'


In [3]:
# For S$P 500 stock

# tickers = pd.read_html( 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# # stock_list = tickers['Symbol'].to_list()
# print(tickers)
# stock_list = tickers['Symbol'][:50]

In [ ]:
def EU_Stock_data():
    """Crawl daily return data from 50 companies in EURO_STOXX_50 with yfinance libary"""

    stock_list = pd.read_html( 'https://en.wikipedia.org/wiki/EURO_STOXX_50')[4]['Ticker'][1:].to_list()
    futures = pd.DataFrame(columns= stock_list)

    time_index = list(yf.Ticker(stock_list[0]).history(period = time_range,start = start_time, end = end_time).index)

    for symbol in stock_list:
        df = yf.Ticker(symbol).history(period = time_range, start = start_time, end = end_time)
        df = pd.DataFrame(df['Close'])
        i = 0
        daily_return = []
        for k in df['Close']:
            if i != 0:
                daily_return.append(float((k-i)/i))
            else:
                daily_return.append(float(0))
            i = k
        if time_index != list(df.index): flag +=1
        try:
            futures[symbol] = daily_return
        except:
            while len(daily_return) < len(futures):
                daily_return.insert(0,float(0))
            futures[symbol] = daily_return

    futures.index = time_index
    futures = futures.iloc[1:, :]

    futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d')
    futures.set_index('Date', inplace=True)

    return futures

In [ ]:
def Volatility_scale(data, ignore_na = False, adjust = True, com = 60, min_periods = 0):
    """Scaling data with votality"""

    
    std_index = data.index

    daily_index = pd.DataFrame(index=std_index)

    for oo in data.columns:
        returns = data[oo]
        returns.dropna(inplace=True, fill = 0)
    
        ret_index = (1 + returns).cumprod()
        ret_index[0] = 1
    
        # ex ante vol estimate
        day_vol = returns.ewm(ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods=min_periods).std(bias=False)
        vol = day_vol * np.sqrt(252) # annualise
    
        ret_index = pd.concat([ret_index, vol], axis=1)
        ret_index.columns = [oo, oo + '_Vol']

        daily_index = pd.concat([daily_index, ret_index], axis=1)

    return daily_index

In [ ]:
def backtest_strategy(data,k, h, vol_flag = 1, target_vol = 0.4, tolerance = 0,ignore_na = False, adjust = True, com = 60, min_periods = 0):
    """Replicate TSMOM strategy"""
    'Input: Data after Crawl'
    '     k lookback time; h holding time'

    "Return:    3 table list include:"
    "               profit and loss daily from 50 companies after using the strategy "
    "               leverage from 50 companies when using the strategy"
    "               signal from 50 companies when using the straegy"

    pnl = pd.DataFrame(index=data.index)
    leverage = pd.DataFrame(index = data.index)
    signal = pd.DataFrame(index = data.index)

    #Scaling data
    daily_index = Volatility_scale(data,ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods=min_periods)

    company = data.columns

    #Volatility settings
    vol_flag = vol_flag               # Set flag to 1 for vol targeting
    if vol_flag == 1:
        target_vol = target_vol # Can be changed
    else:
        target_vol = 'no target vol'
    

    for oo in company:
        flag = 0
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df['returns'] = df[oo].pct_change(k)
        df['pnl'] = 0.
        df['leverage'] = 0.
        df['signal'] = 0.
        for x, v in enumerate(df['returns']):
            if flag != 0:
                # skip the holding return period (since we already calculate it)
                flag = flag - 1
                continue
            if x <= k:
                # skip the first k day(s) observations
                continue
            try:
                if df['returns'].iloc[x-1] < tolerance:
                    for h_period in range(0,h):
                        # negative returns, sell and hold for h day, calculate the return in h day after
                        df['signal'].iloc[x + h_period] = -1
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x - 1 + h_period] / df[oo].iloc[x + h_period] - 1) * \
                                target_vol / df[oo+"_Vol"].iloc[x -1]
                            df['leverage'].iloc[x+h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x - 1 + h_period] / df[oo].iloc[x + h_period] - 1)
                            df['leverage'].iloc[x+h_period] = 1
                elif df['returns'].iloc[x-1] > tolerance:
                    for h_period in range(0,h):
                        # positive returns, buy and hold for h day, calculate the return in h day after
                        df['signal'].iloc[x + h_period] = 1
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1) * \
                                    target_vol / df[oo+"_Vol"].iloc[x - 1]
                            df['leverage'].iloc[x+h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1)
                            df['leverage'].iloc[x+h_period] = 1
            except:pass
            

            # Flag the holding period day
            if df['returns'].iloc[x-1] != tolerance: flag = h - 1



        leverage = pd.concat([leverage, df['leverage']], axis = 1)
        pnl = pd.concat([pnl, df['pnl']], axis=1)
        signal = pd.concat([signal, df['signal']], axis=1)

    pnl.columns = data.columns
    leverage.columns = data.columns
    signal.columns = data.columns


    return [pnl,leverage,signal]

In [ ]:
def strategy_daily_return(pnl):
    
    return pnl.mean(skipna = False, axis=1)